In [1]:
from meza import io
import numpy as np
import pandas as pd

In [17]:
path = '/Users/pierredelice/Library/CloudStorage/Dropbox/Mac/Downloads/Bibliography_tesis_buap/ICOKG_2022/python_bigquery/dgis_datos_id/nacimiento/'
records = io.read(path+'SINAC1036.mdb') # only file path, no file objects
records

<generator object read_mdb at 0x120bc39c0>

In [16]:
from mdb_parser import MDBParser, MDBTable

db = MDBParser(file_path='/Users/pierredelice/Library/CloudStorage/Dropbox/Mac/Downloads/Bibliography_tesis_buap/ICOKG_2022/python_bigquery/dgis_datos_id/nacimiento/SINAC1036.mdb')


In [15]:
# Get and print the database tables
print(db.tables)

# Get a table from the DB.
table = db.get_table("NACIMIENTO")

# Or you can use the MDBTable class.
table = MDBTable(file_path=db, table="NACIMIENTO")

# Get and print the table columns.
print(table.columns)


File not found
Couldn't open database.


CalledProcessError: Command 'mdb-tables /Users/pierredelice/Library/CloudStorage/Dropbox/Mac/Downloads/Bibliography_tesis_buap/ICOKG_2022/python_bigquery/dgis_datos_id/nacimiento/SINAC1036.mdb -d '|'' returned non-zero exit status 1.

In [27]:
table